# *Dijkstra*: Kürzeste Wege in einem Graphen

Die Suche nach kürzesten Verbindungen in Graphen hat in vielen Bereichen des täglichen Lebens praktische Anwendungen:

- **Navigationssysteme** finden den kürzesten Weg zwischen zwei Orten.
- **Lieferunternehmen** suchen die effizienteste Route für die Zustellung von Waren.
- **Stadtplaner und Verkehrsingenieure** möchten die Verkehrsflüsse optimieren, um Staus zu vermeiden.
- **Routenplanung in Netzwerken** zielt darauf, Datenpakete schnell und effizient zu transportieren.

Bereits 1959 entwickelte der niederländische Mathematiker *E. W. Dijkstra* einen Algorithmus, um in gewichteten ungerichteten Graphen kürzeste Wege zu finden.

Natürlich könnte man alle möglichen Wege zwischen zwei Knoten auflisten, um so den kürzesten Weg zu finden (*brute-force-Ansatz*), doch schon in kleinen Graphen gibt es sehr viele solcher Wege, so dass dieses Verfahren nicht wirklich effizient ist.

Der *Dijkstra-Algorithmus* löst das Problem (erstaunlicherweise) demgegenüber sehr effizient.

In diesem Notebook werden wir diesen Algorithmus an einem Beispiel durchführen.

Dazu benutzten wir
- die Programmiersprache **Python** (in der Version 3.10)
- die **Jupyter-Notebook-Umgebung**
- eine spezielle Python-Bibliothek `networkx`, mit der wir sehr leicht gewichtete Graphen implementieren können.
- eine eigene Python-Bibliothek `nrw_graph`, die auf `networkx` basiert, die den Umgang mit Graphen methodisch-didaktisch vereinfacht.

## Begriffsklärung: ***Graphen***

Ein ***Graph*** ist eine abstrakte mathematische Struktur, die aus einer Menge von Knoten (engl.: vertices) und einer Menge von Kanten (engl. edges) besteht. Die Knoten stellen dabei Objekte dar, während die Kanten die Beziehungen zwischen diesen Objekten repräsentieren. 

Graphen werden in vielen Bereichen der Mathematik, Informatik, Ingenieurwissenschaften, Biologie und anderen Wissenschaften verwendet, um komplexe Systeme zu modellieren und zu analysieren. 

Es gibt verschiedene Arten von Graphen:
- gerichtete und ungerichtete Graphen
- gewichtete und ungewichtete Graphen

Die Graphentheorie beschäftigt sich mit der Untersuchung von Graphen und deren Eigenschaften.

Hier sieht man Beispiele von Graphen
- ungerichtet, ungewichtet
- ungerichtet, gewichtet
- gerichtet, ungewichtet
- gerichtet, gewichtet


<td> 
    <img src="Bilder\undir_unweight_graph.png" alt="Drawing" style="width: 200px; float: left;"  hspace=40 /> 
    <img src="Bilder\undir_weight_graph.png" alt="Drawing" style="width: 300px; float: left;" hspace=40/>
    <img src="Bilder\dir_unweight_graph.png" alt="Drawing" style="width: 200px; float: left;"  hspace=40 /> 
    <img src="Bilder\dir_weight_graph.jpeg" alt="Drawing" style="width: 200px; float: left;" hspace=40/>
</td>

In diesem Abschnitt werden ausschließlich ungerichtete, gewichtete Graphen betrachtet.

Weitere Einschränkungen:
- Es gibt in den hier betrachteten Graphen keine *Schlingen* (eine Kante, deren Anfangs- und Endkonten identisch sind).
- Es gibt keine *Mehrfachkanten* (mehrere Kanten zwischen gleichen Knoten).

## Notwendige Bibliotheken importieren

In [ ]:
import networkx as nx
import nrw_graph as ng

# pandas ist eine Bibliothek für Python u.a. zur Verarbeitung von Daten.
import pandas as pd

# Bibliothek, z.B. um Daten graphisch darzustellen.
import matplotlib.pyplot as plt

## Dokumentation der Bibliothek `nrw_graph`

Um Graphenalgorithmen mit Hilfe von Python implementieren zu können, gibt es die Bibliothek `networkx`.

Darauf aufbauend wurde die Bibliothek `nrw_graph` geschrieben, in der einfach zu nutzende Funktionen implementiert sind.

Die Dokumentation der Funktionen dieser Klasse können auf Wunsch eingesehen werden. Dazu müssen Sie den Kommentar der folgende Zelle entfernen, dann die Zelle aktivieren:

In [ ]:
# help("nrw_graph")

## Graph (Kanten, Knoten, Gewicht) aus einer Datei einlesen

Ein *gewichteter Graph* wird beschrieben durch die Angabe der zu dem Graphen gehörenden Kanten. Eine Kante ist dabei ein Objekt, in dem die Namen der beiden Endknoten sowie das Gewicht der Kante (in unserem Beispiel die Länge) enthalten sind.

Da die Knoten, die der Graph enthält, nicht explizit angegeben werden, sondern sich aus den Endknoten der Kanten ergeben, kann man auf diese Weise keine Graphen mit isolierten Knoten erzeugen. Jedoch ist das für unser Beispiel nicht tragisch, da von und zu isolierten Knoten sicherlich kein Weg führt.

Die für einen Graphen notwendigen Daten sollten sich in einer CSV-Datei befinden. Eine solche Datei enthält die Daten (also die Information über eine Kante) zeilenweise, wobei die erste Zeile ein Art Überschrift ist.

Jede Zeile enthält - in der Regel duch Kommata oder Semikolon getrennt - die Werte der jeweiligen Attribute:

- Name des Startknoten
- Name des Zielknoten
- Länge der Kante

Dabei sind in diesem Zusammenhang die Begriffe *Start* und *Ziel* ggf. missverständlich, da die Graphen, die hier benutzt werden, ungerichtet sind; gibt es also eine Kante von A nach B, die in dem Datensatz  angegeben ist, gibt es automatisch auch die Kante von B nach A gleicher Länge, ohne dass sie explizit in den Datensätzen auftaucht.

In [ ]:
#df_staedte = pd.read_csv("staedte.txt", sep=",")
df_staedte = pd.read_csv("stdt.txt", sep=",")

# Hier werden aus Gründen der Übersichtlichkeit 
# nur die ersten 10 Datensätze in einer Tabelle gezeigt.
display(df_staedte.head(10)) #

## Der Graph wird aus den Daten konstruiert

In [ ]:
# Ein neuer leerer Graph
autobahn = ng.nrw_graph()

zeilen = df_staedte.shape[0]
for i in range(zeilen):
    source = df_staedte.iloc[i]['Start']
    target = df_staedte.iloc[i]['Ziel']
    dist = float(df_staedte.iloc[i]['Entfernung'])

    autobahn.fuegeKanteHinzu(source, target, gewicht=dist)
    autobahn.deflagKnoten(source)
    autobahn.deflagKnoten(target)

## Zeig mal den Graphen

Wenn man möchte, kann man den Graphen visualisieren. 

Doch **Vorsicht**: 

- Die Daten in der Datei enthalten keine Angaben über die Lage der Knoten zueinander. Damit ein einigermaßen realistisches Bild der Autobahnverbindungen entstehen kann, wurden in dem folgenden Python-Programm die Längen- und Breitengrade der Städte in Form eines Dictionaries angegeben.

Diese Darstellung ist nur eine nette Spielerei, um die Fähigkeit der Bibliothek zu demonstrieren! In den folgenden Abschnitten, in denen es um kürzeste Verbindungen geht, spielen diese Angaben keine Rolle mehr. 

In [ ]:
# explicitly set positions
# (Längengrad östl. , Breitengrad nördl.)
pos = {'KI' : (10.12, 54.32), 
       'SN' : (11.40, 53.63), 
       'HH' : ( 9.99, 53.55), 
       'HB' : ( 8.80, 53.07), 
       'BI' : ( 8.53, 52.03), 
       'H'  : ( 9.73, 52.37), 
       'MD' : (11.62, 52.12), 
       'B'  : (13.37, 52.51), 
       'D'  : ( 6.77, 51.22), 
       'MZ' : ( 8.24, 49.99), 
       'EF' : (11.02, 50.98), 
       'DD' : (13.73, 51.05), 
       'SB' : ( 6.99, 49.24), 
       'S'  : ( 9.18, 48.77), 
       'M'  : (11.57, 48.13), 
       'HAM': (7.81, 51.67), 
       
# Die folgenden Einträge benutzen die korrekten Werte: 
#       'P'  : (13.06, 52.39), # orig
#       'MS' : ( 7.62, 51.96), # orig
#       'WI' : ( 8.23, 50.07), # orig 

# Die folgenden Einträge benutzen leicht verschobene Werte, 
# damit sich die Knoten in der Graphik nicht überlappen!
       'P'  : (12.90, 52.39), 
       'MS' : ( 7.62, 52.20), 
       'WI' : ( 8.23, 50.60),  
      }

node_options = {
    "node_color": "yellow",
    "edgecolors": "black",
    "node_size": 290,
    "linewidths": 1,
                }
edge_options = {
    "edge_color": "blue",
    "width": 1,
}

label_options = {
    "font_size": 6, 
    "font_color" : "black",
}


# nodes:
nx.draw_networkx_nodes(autobahn, pos, **node_options)

# edges:
nx.draw_networkx_edges(autobahn, pos, **edge_options)

# labels:
nx.draw_networkx_labels(autobahn, pos, **label_options)

ax = plt.gca()
ax.set_xlabel("östl. Länge")
ax.set_ylabel("nördl. Breite")
ax.set_title("Städte in Deutschland")

ax.margins(0.01)
plt.axis("on")
plt.show()

## Kontrolle: Hat das Einlesen der Daten geklappt?

In [ ]:
autobahn.alleKanten()

In [ ]:
for kante in autobahn.alleKanten():
    start = kante[0]
    ziel = kante[1]
    print(start.ljust(12), " -- ", ziel.ljust(12), ":", autobahn.kantenGewicht(start, ziel))

## Einige Hilfsfunktionen

### Informationen über Knoten; Markieren von Knoten

Wir betrachten einmal einen Knoten K des Graphen. In vielen Situationen ist es wichtig zu wissen, von welchem Knoten V (der Vorgängerknoten von K) man zu K kommt und wie weit es dabei ist. 
- Manchmal möchte man wissen, wie weit der es von V zu K ist, 
- in anderen Fällen ist die Entfernung vom Start über V zu K interessant.

Insgesamt kann man die drei Informationen 
- Name von K
- Name des Vorgängerknoten
- Entfernung zu K

als 3-elementige Liste verwalten, die als Information genutzt wird.

### Knoten können *besucht* werden

Knoten können ein boolsches Flag (`True` oder `False`) haben. 

Ein Knoten K nennen wir *besucht*, wenn bekannt ist, wie lang der kürzeste Weg vom Start zu K ist. 
In einem Graphen sind zunächst alle Knoten unbesucht, haben also die Flagge `False`.

Ein besuchter Knoten hat die Flagge `True`. Wenn Knoten besucht sind, haben sie eine Marke in Form einer 3-elementigen Liste (s.o.).

Die folgende Funktion erzeugt eine Liste aller besuchten Knoten (genauer der zugehörigen Knotenmarken):

In [ ]:
def alleBesuchtenKnoten():
    alle = []
    for knoten in autobahn.alleKnoten():
        if autobahn.knotenHatFlag(knoten):
            (ueber, lang) = autobahn.getKnotenMarke(knoten)
            alle.append([knoten,ueber, lang])
    return alle

In [ ]:
# Eine Hilfsfunktion, damit man die später die Liste von Kanten (s.o.) sortieren kann.
# Die einträge in einer solchen Liste sind ebenfalls Listen der Form [über,ziel,entfernung] 
def entfernung (liste):
    return liste[2]

### Wir finden *Folgeknoten*

Für jeden Knoten K im Graphen ist es wichtig zu wissen, welche Knoten F von K aus direkt zu erreichen sind. Dabei werden die drei Informationen
- Name von K
- Name von F
- Länge der Kante K-F

in Form einer 3-elementigen Liste (s.o.) verwaltet.

Die folgende Funktion liefert eine Liste aller möglichen Kanten von K aus zu Folgeknoten (bzw. deren Kanteninfos):

In [ ]:
def kantenVon(von):
    kanten = []
    for (s,z) in autobahn.alleKanten():
        if s == von:
            l = autobahn.kantenGewicht(s, z)
            kanten.append([s, z, l])
        elif z == von:
            l = autobahn.kantenGewicht(s, z)
            kanten.append([z, s, l])

    return kanten

### Wir finden *lokale Schnittkanten*

Betrachten wir jetzt einen bereits besuchten Knoten K.

Im Gegensatz zur vorigen Funktion `kantenVon` inetressieren wir uns jetzt nur für solche Folgeknoten F, die noch nicht besucht sind. Eine Kante von K zu dem unbesuchten Folgeknoten F nennen wir eine *lokale Schnittkante*.

Zu dem Folgeknoten F haben wir also (erneut in Form einer 3-elementigen Liste) die Informationen:
- Name von K
- Name von F
- Länge der Kante K-F

Die folgende Funktion liefert eine Liste aller möglichen Schnittkanten von K aus zu Folgeknoten (bzw. deren Kanteninfos):

In [ ]:
def lokaleSchnittkantenVon(von):
    kanten = []
    for (s,z) in autobahn.alleKanten():
        if s == von and not autobahn.knotenHatFlag(z):
            l = autobahn.kantenGewicht(s, z)
            kanten.append([s, z, l])
        elif z == von and not autobahn.knotenHatFlag(s):
            l = autobahn.kantenGewicht(s, z)
            kanten.append ([z,s, l])

    return kanten

### Wir finden *Schnittkanten*

Die Schnittkanteninformationen, die wir in der vorigen Funktion erzeugt haben, sind jedoch für unsere Zwecke nicht zielführend, da zu einem Folgeknoten F von K nicht die Kantenlänge K-F, sondern die Weglänge vom Start über K zu F wichtig ist.

Eine solche Kante nennen wir *Schnittkante*.

Also erzeugen wir zu jedem unbesuchten Folgeknoten F die Informationen

- Name von K
- Name von F
- Länge des Weges vom Start über K zu F

Die folgende Funktion erzeugt eine Liste aller Schnittkanteninformationen im Graph. Es werden also zu allen besuchten Knoten K die Schnittkanteninfos erzeugt:

In [ ]:
def alleSchnittkanten():
    kanten = []
    for (s,z) in autobahn.alleKanten():
        if autobahn.knotenHatFlag(s) and not autobahn.knotenHatFlag(z):
            l = autobahn.kantenGewicht(s, z)
            (ueber, weit) = autobahn.getKnotenMarke(s)
            l += weit
            kanten.append ([s, z, l])
        elif autobahn.knotenHatFlag(z) and not autobahn.knotenHatFlag(s):
            l = autobahn.kantenGewicht(z, s)
            (ueber, weit) = autobahn.getKnotenMarke(z)
            l += weit
            kanten.append([z, s, l])

    return kanten

### Markierung von besuchten Knoten

Wenn ein Knoten K besucht ist, möchte man den kürzesten Weg vom Start zu K kennen. 

Kennt man zu jedem besuchten Knoten den Vorgänger auf dem kürzesten Weg, kann der kürzeste Weg rückwärts rekonstruiert werden.

Also markieren wir jeden besuchten Knoten K mit einem Tupel, bestehend aus:

- Name des Vorgängers V
- Länge des kürzesten Weges vom Start über V zu K

## Dijkstra "Zu Fuß" lösen: Von Berlin nach München

In [ ]:
startknoten = "B"
zielknoten = "M"

### Berlin ist bereits besucht!

Zunächst eine Trivialität:
- Möchte man von Berlin nach Berlin reisen, so ist die kürzeste Verbindung über Berlin mit einer Länge von 0.0

Also wird Berlin mit einer Flagge und eine Marke der Form (ueber, laenge) versehen:

In [ ]:
autobahn.flagKnoten("B")
autobahn.markiereKnoten("B", ("B", 0.0))

Jetzt kann man sich alle geflaggten Knoten mit ihren Marken ansehen.

**Schau dir dazu die entsprechende Hilfsfunktion weiter oben an!**

In [ ]:
alleBesuchtenKnoten()

### Welcher Ort ist Berlin am nächsten?

**Schau dir auch dazu die entsprechende Hilfsfunktion weiter oben an!**

In [ ]:
kantenVon("B")

Also ist Potsdam derjenige Ort, der von Berlin am nächsten liegt, so dass wir
Potsdam als besucht betrachten können.

Will man also von Berlin nach Potsdam, dann (mal wieder eine Trivialität) fährt man über Berlin; die Strecke hat
eine Länge von 35.0

Diese Informationen trägt man ein:

In [ ]:
autobahn.flagKnoten("P")
autobahn.markiereKnoten("P", ("B", 35.0))

Zur Kontrolle:

In [ ]:
alleBesuchtenKnoten()

### Jetzt geht's weiter: von Berlin oder von Potsdam?

Man kann jetzt entweder 
- von Berlin aus direkt 
- oder von Berlin über Potsdam 

weiterfahren zu einem Ort, der möglichst nahe ist.

***Definition*** Kanten, die einen besuchten mit einem unbesuchten Ort verbinden, nennt man **Schnittkanten**

Also suchen wir zunächst alle Orte (mitsamt Entfernungen), die von Berlin direkt erreichbar sind. 
Dabei lassen wir natürlich den bereits besuchten Ort Potsdam aus:

In [ ]:
lokaleSchnittkantenVon("B")

Jedoch müssen wir auch Schnittkanten - ausgehend von Potsdam - betrachten. Dabei ist aber zu beachten, dass ein Ort X, der von Potsdam direkt erreichbar ist, eine Gesamtroute der Form 

- Berlin - Potsdam - X

hat, so dass die Weglänge sich dann zusammensetzt aus der Länge von (Berlin - Potsdam) und der Länge (Potsdam - X).

In [ ]:
alleSchnittkanten()

Das ergibt also insgesamt 5 Schnittkanten:

1. 'Berlin' - 'Schwerin', 224.0,
1. 'Potsdam' - 'Hannover', 299.0,
1. 'Potsdam' - 'Erfurt', 316.0,
1. 'Potsdam' - 'Magdeburg', 169.0,
1. 'Berlin' - 'Dresden', 193.0

und damit 5 Routen von Berlin aus:

1. 'Berlin', 'Schwerin', 224.0,
1. 'Berlin' - 'Potsdam - 'Hannover', 299.0,
1. 'Berlin' - 'Potsdam' - 'Erfurt', 316.0,
1. 'Berlin' - 'Potsdam' - 'Magdeburg', 169.0,
1. 'Berlin', 'Dresden', 193.0


Die Route nach Magdeburg (über Potsdam) ist also die kürzeste. Das müssen wir jetzt eintragen:

In [ ]:
autobahn.flagKnoten("MD")
autobahn.markiereKnoten("MD", ("P", 169.0))

Auch hier die Kontrolle:

In [ ]:
alleBesuchtenKnoten()

### Jetzt ist alles klar!?

***Aufgabe***: 

Setze das Verfahren fort.

## Ab hier die schrittweise Lösung

***Also bitte nur ansehen, falls nötig!***

In [ ]:
alleSchnittkanten()

In [ ]:
autobahn.flagKnoten("DD")
autobahn.markiereKnoten("DD", ("B", 193.0))

In [ ]:
alleBesuchtenKnoten()

In [ ]:
alleSchnittkanten()

In [ ]:
autobahn.flagKnoten("SN")
autobahn.markiereKnoten("SN", ("B", 224.0))

In [ ]:
alleBesuchtenKnoten()

In [ ]:
alleSchnittkanten()

In [ ]:
autobahn.flagKnoten("H")
autobahn.markiereKnoten("H", ("P", 299.0))

In [ ]:
alleBesuchtenKnoten()

In [ ]:
alleSchnittkanten()

In [ ]:
autobahn.flagKnoten("EF")
autobahn.markiereKnoten("EF", ("P", 316.0))

In [ ]:
alleBesuchtenKnoten()

In [ ]:
alleSchnittkanten()

In [ ]:
autobahn.flagKnoten("HH")
autobahn.markiereKnoten("HH", ("SN", 334.0))

In [ ]:
alleBesuchtenKnoten()

In [ ]:
alleSchnittkanten()

In [ ]:
autobahn.flagKnoten("KI")
autobahn.markiereKnoten("KI", ("SN", 384.0))

In [ ]:
alleBesuchtenKnoten()

In [ ]:
alleSchnittkanten()

In [ ]:
autobahn.flagKnoten("BI")
autobahn.markiereKnoten("BI", ("H", 389.0))

In [ ]:
alleBesuchtenKnoten()

In [ ]:
alleSchnittkanten()

In [ ]:
autobahn.flagKnoten("HB")
autobahn.markiereKnoten("HB", ("H", 426.0))

In [ ]:
alleBesuchtenKnoten()

In [ ]:
alleSchnittkanten()

In [ ]:
autobahn.flagKnoten("HAM")
autobahn.markiereKnoten("HAM", ("BI", 469.0))

In [ ]:
alleBesuchtenKnoten()

In [ ]:
alleSchnittkanten()

In [ ]:
autobahn.flagKnoten("MS")
autobahn.markiereKnoten("MS", ("HAM", 539.0))

In [ ]:
alleBesuchtenKnoten()

In [ ]:
alleSchnittkanten()

In [ ]:
autobahn.flagKnoten("D")
autobahn.markiereKnoten("D", ("HAM", 579.0))

In [ ]:
alleBesuchtenKnoten()

In [ ]:
alleSchnittkanten()

In [ ]:
autobahn.flagKnoten("WI")
autobahn.markiereKnoten("WI", ("EF", 599.0))

In [ ]:
alleBesuchtenKnoten()

In [ ]:
alleSchnittkanten()

In [ ]:
autobahn.flagKnoten("MZ")
autobahn.markiereKnoten("MZ", ("WI", 607.0))

In [ ]:
alleBesuchtenKnoten()

In [ ]:
alleSchnittkanten()

In [ ]:
autobahn.flagKnoten("M")
autobahn.markiereKnoten("M", ("DD", 653.0))

In [ ]:
alleBesuchtenKnoten()

### Wir sind in München angekommen!

Wir können jetzt den Weg von Berlin nach München erkennen, indem wir quasi rückwärts laufen:

- von Dresden nach München
- von Berlin nach Dresden

Insgesamt hat der kürzeste Weg Berlin - Dresden - München eine Länge von 653.0 km

### Wir können auch den besten Weg von Berlin nach Münster finden:

- von Hamm nach Münster
- von Bielefeld nach Hamm
- von Hannover nach Bielefeld
- von Potsdfam nach Hannover
- von Berlin nach Postdam

Nach Münster sind es also 539 km:

Berlin - Potsdam - Hannover - Bielefeld - Hamm - Münster